In [23]:
import pickle
import os
import sys

import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table
import astropy.units as u
from astropy.cosmology import WMAP9
import astropy.constants as c

from matplotlib import pyplot as plt
%matplotlib

import spectroscopy

Using matplotlib backend: Qt5Agg


In [2]:
FIG_DIR = '../figures'

In [3]:
sys.path.append('../code')
import scale_merge_spec

In [4]:
from pyraf import iraf
from iraf import noao,onedspec,scombine as scombine
from iraf import noao,onedspec,rspectext as rspectext

In [5]:
def scale_spectra(sn_dir, combine_spec_dict):
    for date in combine_spec_dict.keys():
        if combine_spec_dict[date]['scombine'] is True:
            date_dict = combine_spec_dict[date]

            #Find which spectrum to scale all the other spectra to
            overlap_wmin = []
            overlap_wmax = []
            template_spec = scale_merge_spec.read_spec(sn_dir, date_dict['filename'][0])
            spec_list = [template_spec]
            for ifile in date_dict['filename'][1:]:
                spec = scale_merge_spec.read_spec(sn_dir, ifile)
                wmin, wmax = scale_merge_spec.find_overlap_values(template_spec, spec)
                overlap_wmin.append(wmin)
                overlap_wmax.append(wmax)
                spec_list.append(spec)
            mutual_overlap = [max(overlap_wmin), min(overlap_wmax)]
            max_flux_spec = None
            for ispec in spec_list:
                #Net counts spectra can't be the spectra we scale to
                if (ispec.flux<1E-5).any():
                    overlap_flux = ispec.flux[(ispec.wavelength<mutual_overlap[1])&(ispec.wavelength>mutual_overlap[0])]
                    if max_flux_spec is None:
                        max_flux_spec = ispec
                        max_flux = np.max(overlap_flux)
                    else:
                        if np.max(overlap_flux) > max_flux:
                            max_flux = np.max(overlap_flux)
                            max_flux_spec = ispec
            #Scale spectra and write in scombine format; make scombine list
            ofile = open('{}_scomb_list.txt'.format(date), 'w')
            for ispec in spec_list:
                print(ispec.filename)
                spec1, spec2 = scale_merge_spec.scale_spectrum(max_flux_spec, ispec, spectrum_to_scale=2)
                tbdata = Table([spec2.wavelength, spec2.flux], names=['wavelength', 'flux'])
                ascii_filename = '{}.txt'.format(os.path.splitext(ispec.filename)[0])
                fits_filename = '{}_scaled.fits'.format(os.path.splitext(ispec.filename)[0],)
                tbdata.write(ascii_filename, format='ascii.commented_header', overwrite=True)
                if os.path.exists(fits_filename):
                    os.remove(fits_filename)
                rspectext(input=ascii_filename, 
                          output=fits_filename, 
                          flux='yes',
                          crval1=ispec.wavelength[0], 
                          cdelt1=ispec.wavelength[1]-ispec.wavelength[0])
                os.remove(ascii_filename)
                ofile.write('{}\n'.format(fits_filename))
            ofile.close()

In [28]:
def scombine_spec(sn, combine_spec_dict, redshift):
    for date in combine_spec_dict.keys():
        if combine_spec_dict[date]['scombine'] is True:
            input_file = '{}_scomb_list.txt'.format(date)
            output_file = '../SNID_templates/{}.fits'.format(date)
            if os.path.exists(output_file):
                os.remove(output_file)
            scombine(input='@{}'.format(input_file), output=output_file)
            os.remove(input_file)
            spec = scale_merge_spec.read_spec(*os.path.split(output_file))
            #deredshift
            #spec.wavelength = spectroscopy.apply_redshift(spec.wavelength, redshift)
            tbdata = Table([spec.wavelength, spec.flux])
            tbdata.write('../SNID_templates/{}_{}_combine.dat'.format(sn,os.path.split(output_file)[1][0:10]), 
                         overwrite=True,
                         format='ascii.fixed_width_no_header',
                        delimiter=' ',
                         delimiter_pad=None,
                         bookend=False,
                        formats={'col0':'{0: >13.2F}', 'col1':'{: >13.6E}'})
            os.remove(output_file)

# Test Output

In [19]:
def plot_combined_spec(sn_dir, sn, combine_spec_dict):
    for date in combine_spec_dict.keys():
        if combine_spec_dict[date]['scombine'] is True:
            plt.figure()
            date_dict = combine_spec_dict[date]
            combine_spec = scale_merge_spec.read_spec('../SNID_templates/','{}_{}_combine.dat'.format(sn,date))
            for ifile in date_dict['filename']:
                spec = scale_merge_spec.read_spec(sn_dir, ifile)
                junk, scale_spec = scale_merge_spec.scale_spectrum(combine_spec, spec, spectrum_to_scale=2)
                plt.plot(scale_spec.wavelength, scale_spec.flux, label=ifile)
            plt.plot(combine_spec.wavelength, combine_spec.flux, color='k', label='Combined')
            plt.legend(loc='best', fontsize='x-small')
            plt.xlabel('wavelength')
            plt.ylabel('flux')
            plt.title('Combined Spectra for {} {}'.format(sn, date))
            plt.savefig(os.path.join(FIG_DIR, '{}_{}_combined.pdf'.format(sn, date)))
            plt.close()

# 2013ej

In [29]:
combine_spec_dict = {'2013-07-27':{'filename':('2013ej_20130727_fts.fits'), 'scombine':False},
                    '2013-07-31':{'filename':('2013ej_20130731_P122_BC_300tr.fits'), 'scombine':False},
                    '2013-08-01':{'filename':('2013ej_20130801_ftn.fits'), 'scombine':False},
                    '2013-08-02':{'filename':('2013ej_20130802_ftn.fits',
                                              '2013ej_20130802_fts.fits'), 'scombine':False},
                    '2013-08-03':{'filename':('2013ej_20130803_fts.fits'), 'scombine':False},
                    '2013-08-04':{'filename':('sn2013ej-20130804.383-r.flm', 
                                              '2013ej_20130804_fts.fits',
                                              'sn2013ej-20130804.511-ui.flm'), 'scombine':True},
                    '2013-08-05':{'filename':('2013ej_20130805_ftn.fits',
                                              '2013ej_20130805_fts.fits',
                                              'SN2013ej_20130804_Gr11_Free_slit1.0_56999_1.fits',
                                              'SN2013ej_20130804_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':True},
                    '2013-08-07':{'filename':('2013ej_20130807_P122_BC_300tr.fits'), 'scombine':False},
                    '2013-08-08':{'filename':('2013ej_20130808_fts.fits',
                                              '2013ej_20130808_P122_BC_300tr.fits'), 'scombine':True},
                    '2013-08-09':{'filename':('2013ej_20130809_fts.fits'), 'scombine':False},
                    '2013-08-10':{'filename':('2013ej_20130810_fts.fits'), 'scombine':False},
                    '2013-08-11':{'filename':('2013ej_20130811_fts.fits', 
                                              'SN2013ej_20130811_wifes_B.fits',
                                              'SN2013ej_20130811_wifes_R.fits'), 'scombine':True},
                    '2013-08-12':{'filename':('2013ej_20130812_fts.fits',
                                              'sn2013ej-20130812.508-ui.flm'), 'scombine':True},
                    '2013-08-13':{'filename':('2013ej_20130813_fts.fits'), 'scombine':False},
                    '2013-08-15':{'filename':('2013ej_20130815_fts.fits',
                                              'SN2013ej_20130814_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':True},
                     #Spectra shape inconsistent with day before and day after
                    #'2013-08-16':{'filename':('SN2013ej_20130815_Gr11_Free_slit1.0_56999_1.fits',
                    #                          'SN2013ej_20130815_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-08-17':{'filename':('2013ej_20130817_fts.fits'), 'scombine':False},
                    '2013-08-19':{'filename':('SN2013ej_20130819_wifes_B.fits',
                                              'SN2013ej_20130819_wifes_R.fits'), 'scombine':False},
                    '2013-08-27':{'filename':('SN2013ej_20130826_Gr11_Free_slit1.0_56999_1.fits',
                                              'SN2013ej_20130826_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-08-30':{'filename':('SN2013ej_20130829_Gr11_Free_slit1.0_56999_1.fits',
                                              'SN2013ej_20130829_Gr16_OG530_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-09-06':{'filename':('sn2013ej-20130906.414-ui.flm'), 'scombine':False},
                    '2013-09-09':{'filename':('SN2013ej_20130908_Gr11_Free_slit1.5_56999_1.fits',
                                              'SN2013ej_20130908_Gr16_OG530_slit1.5_56999_1.fits'), 'scombine':False},
                    '2013-09-10':{'filename':('sn2013ej-20130910.598-hal.flm'), 'scombine':False},
                    '2013-09-13':{'filename':('SN2013ej_20130912_Gr11_Free_slit1.0_56999_1.fits'), 'scombine':False},
                    '2013-09-19':{'filename':('SN2013ej_20130919_wifes_B.fits',
                                              'SN2013ej_20130919_wifes_R.fits'), 'scombine':False},
                    '2013-10-01':{'filename':('sn2013ej-20131001.548-ui.flm'), 'scombine':False},
                    '2013-10-03':{'filename':('SN2013ej_20131002_Gr11_Free_slit1.0_57001_1.fits',
                                              'SN2013ej_20131002_Gr16_OG530_slit1.0_57001_1.fits'), 'scombine':False},
                    '2013-10-05':{'filename':('sn2013ej-20131005.329-ui.flm'), 'scombine':False},
                    '2013-10-08':{'filename':('sn2013ej-20131008.483-hal.flm'), 'scombine':False},
                    '2013-10-10':{'filename':('sn2013ej-20131010.291-ui.flm'), 'scombine':False},
                    '2013-10-16':{'filename':('SN2013ej_20131016_wifes_B.fits',
                                              'SN2013ej_20131016_wifes_R.fits'), 'scombine':False},
                    '2013-10-24':{'filename':('SN2013ej_20131024_wifes_B.fits',
                                              'SN2013ej_20131024_wifes_R.fits'), 'scombine':False},
                    '2013-10-26':{'filename':('sn2013ej-20131026.264-ui.flm'), 'scombine':False},
                    '2013-10-27':{'filename':('SN2013ej_20131026_Gr11_Free_slit1.0_57001_1.fits',
                                              'SN2013ej_20131026_Gr16_OG530_slit1.0_57001_1.fits'), 'scombine':False},
                    '2013-11-02':{'filename':('sn2013ej-20131102.341-ui.flm'), 'scombine':False},
                    '2013-11-08':{'filename':('sn2013ej-20131108.312-ui.flm'), 'scombine':False},
                    '2013-11-24':{'filename':('SN2013ej_20131123_Gr11_Free_slit1.0_57002_1.fits',
                                              'SN2013ej_20131123_Gr16_OG530_slit1.0_57002_1.fits'), 'scombine':False},
                    '2013-11-28':{'filename':('sn2013ej-20131128.367-ui.flm'), 'scombine':False},
                    '2013-12-06':{'filename':('sn2013ej-20131206.394-ui.flm'), 'scombine':False},
                    '2013-12-24':{'filename':('SN2013ej_20131223_Gr11_Free_slit1.0_57002_1.fits',
                                              'SN2013ej_20131223_Gr16_OG530_slit1.0_57002_1.fits'), 'scombine':False},
                    '2014-01-24':{'filename':('SN2013ej_20140123_Gr11_Free_slit1.0_57003_1.fits'), 'scombine':False},
                    '2014-01-31':{'filename':('SN2013ej_20140130_Gr16_OG530_slit1.5_57003_1.fits'), 'scombine':False}}

In [30]:
sn_dir = '../2013ej'
sn='sn13ej'
dist = 9.7*u.Mpc
redshift = WMAP9.H0*dist/c.c.to(u.km/u.s)
print(redshift)

0.00224289831868


In [31]:
scale_spectra(sn_dir, combine_spec_dict)
scombine_spec(sn, combine_spec_dict, redshift)

2013ej_20130808_fts.fits
2013ej_20130808_P122_BC_300tr.fits
2013ej_20130815_fts.fits
SN2013ej_20130814_Gr16_OG530_slit1.0_56999_1.fits
sn2013ej-20130804.383-r.flm
2013ej_20130804_fts.fits
sn2013ej-20130804.511-ui.flm
2013ej_20130805_ftn.fits
2013ej_20130805_fts.fits
SN2013ej_20130804_Gr11_Free_slit1.0_56999_1.fits
SN2013ej_20130804_Gr16_OG530_slit1.0_56999_1.fits
2013ej_20130811_fts.fits
SN2013ej_20130811_wifes_B.fits
SN2013ej_20130811_wifes_R.fits
2013ej_20130812_fts.fits
sn2013ej-20130812.508-ui.flm

Sep 21 15:41: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  2013ej_20130808_fts_scaled.fits[  1]
  2013ej_20130808_P122_BC_300tr_scaled.fits[  1]

  Output image = ../SNID_templates/2013-08-08.fits, ncombine = 2
  w1 = 3150.063, w2 = 9980.93, dw = 1.738134, nw = 3931., dtype = 0

Sep 21 15:41: SCOMBINE
  combine = average, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  2013ej_20130815_fts_sca

In [32]:
plot_combined_spec(sn_dir, sn, combine_spec_dict)